# <center>**Práctica 2: Trabajo de Investigación**</center>
## <center>Procesamiento de Imagen, Vídeo y Audio</center>
### <center>Grado en Ciencia e Ingeniería de Datos</center>
### <center>Curso 2022/2023</center>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import skimage.io as io
import scipy as sp
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

Unet

Igual si hay tiempo es buena idea ponerlo como prueba también  

balancear imagenes 50% negro 50% blanco - para que detecte las carreteras
quitar pixeles negros para tener 50-50

mascara como y 
rgb como data

### **1. Ejercicio 1: Segmentación de carreteras en imagen aérea de alta resolución**

In [ ]:
# cargamos los datos de la carpeta data 
data = io.imread_collection('/content/drive/MyDrive/piva-p2/data/roads/sat/*.tiff')
labels = io.imread_collection('/content/drive/MyDrive/piva-p2/data/roads/gt/*.tif')

MEJOR HACER UN BASELINE MAS SENCILLO, Y MAS ADELANTE COMPARAR MODELOS Y ASI DECIDIR QUE NOS QUEDAMOS CON LA U-NET

In [ ]:
# utilizar np.split para obtener train, val y test
X_train, X_val, X_test = np.split(data, [int(.7*len(data)), int(.9*len(data))])
y_train, y_val, y_test = np.split(labels, [int(.7*len(labels)), int(.9*len(labels))])

y_train = np.expand_dims(y_train, axis=3)
y_val = np.expand_dims(y_val, axis=3)
y_test = np.expand_dims(y_test, axis=3)

In [ ]:
# hacemos un baseline autoencoder para ver que tal se comporta
def baseline(input_shape):
    # encoder
    input_img = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

    # decoder
    x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = tf.keras.models.Model(input_img, decoded)

    return autoencoder

In [ ]:
baseline = baseline([1500,1500,3])
baseline.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()]) #['accuracy', 'mse', 'mae', 'mape'])

In [ ]:
baseline.fit(X_train, y_train, epochs=5, batch_size=1, validation_data=(X_val, y_val))

In [ ]:
baseline.evaluate(X_test, y_test)

predictions = baseline.predict(X_test)

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(y_test[0], cmap='gray')
ax1.set_title('GT image')
ax1.axis('off')
ax2.imshow(predictions[0], cmap='gray')
ax2.set_title('Predicted image')
ax2.axis('off')
plt.show()

U-NET

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, Concatenate, BatchNormalization, Conv2DTranspose
from tensorflow.keras.models import Model

def unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    batch1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(3,3))(batch1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    batch2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(batch2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    batch3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(batch3)  

    # Bottleneck
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    batch4 = BatchNormalization()(conv4)

    # Decoder
    convtrans1 = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(batch4)
    merge1 = Concatenate(axis=3)([convtrans1, batch4])
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge1)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

    up1 = UpSampling2D(size=(2,2))(conv5)
    convtrans2 = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(up1)
    merge2 = Concatenate(axis=3)([convtrans2, batch3])
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge2)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

    up2 = UpSampling2D(size=(2,2))(conv6)
    convtrans3 = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(up2)
    merge3 = Concatenate(axis=3)([convtrans3, batch2])
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(merge3)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

    up3 = UpSampling2D(size=(3,3))(conv7)
    convtrans4 = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(up3)
    merge4 = Concatenate(axis=3)([convtrans4, batch1])

    # Output
    outputs = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(merge4)

    model = Model(inputs, outputs)

    return model

In [ ]:
U_net = unet((1500, 1500, 3))
U_net.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()]) #['accuracy', 'mse', 'mae', 'mape'])

In [ ]:
U_net.fit(X_train, y_train, epochs=5, batch_size=1, validation_data=(X_val, y_val))

In [ ]:
# Evaluamos
U_net.evaluate(X_test, y_test)

In [ ]:
predictions = U_net.predict(X_test)

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(y_test[0], cmap='gray')
ax1.set_title('GT image')
ax1.axis('off')
ax2.imshow(predictions[0], cmap='gray')
ax2.set_title('Predicted image')
ax2.axis('off')
plt.show()


#### Preprocesado de las imágenes

In [ ]:
from skimage import exposure

# Normalización de imágenes
norm_data = []
for img in data:
    norm_img = exposure.equalize_adapthist(img, clip_limit=0.03) # Normaliza la imagen usando la corrección de contraste adaptativo
    norm_data.append(norm_img)

# Visualización de una imagen normalizada
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(data[0])
ax1.set_title('Imagen original')
ax2.imshow(norm_data[0])
ax2.set_title('Imagen normalizada')
plt.show()

In [ ]:
from skimage.filters import gaussian, median

# Eliminación de ruido 
smooth_data = []
for img in norm_data:
    smooth_img = gaussian(img, sigma=1) # Filtro Gaussiano con sigma=1
    smooth_img = median(smooth_img) # Filtro de mediana con tamaño de ventana predeterminado
    smooth_data.append(smooth_img)

# Visualización de una imagen suavizada
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(norm_data[0])
ax1.set_title('Imagen normalizada')
ax2.imshow(smooth_data[0])
ax2.set_title('Imagen suavizada')
plt.show()

In [ ]:
from skimage.transform import resize

# Ajuste de la resolución de las imágenes
resized_data = []
for i in range(len(smooth_data)):
    resized_img = resize(smooth_data[i], (512, 512), anti_aliasing=True)
    resized_data.append(resized_img)

# Visualización de una imagen redimensionada
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(smooth_data[0])
ax1.set_title('Imagen suavizada')
ax2.imshow(resized_data[0])
ax2.set_title('Imagen redimensionada')
plt.show()

#### Extracción de características

In [ ]:
# Segmentación de color de las imágenes
from skimage.color import rgb2hsv
from skimage.filters import threshold_otsu

segmented_data = []
for img in resized_data:
    hsv_img = rgb2hsv(img)
    v = hsv_img[:, :, 2]
    thresh = threshold_otsu(v)
    binary = v > thresh
    segmented_data.append(binary)

# Visualización de una imagen segmentada
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(resized_data[0])
ax1.set_title('Imagen redimensionada')
ax2.imshow(segmented_data[0])
ax2.set_title('Imagen segmentada')
plt.show()

In [ ]:
# convertimos resized_data a grayscale
gray_resized_data = [np.dot(img[...,:3], [0.299, 0.587, 0.114]) for img in resized_data]

In [ ]:
from skimage import feature

# Detección de bordes utilizando el operador Canny
# convertir resized_data a escala de grises
edges = []
for i in range(len(gray_resized_data)):
    edge = feature.canny(gray_resized_data[i], sigma=4, low_threshold=0.05, high_threshold=0.2)
    edges.append(edge)

# Visualización de la imagen original y los bordes detectados
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(gray_resized_data[2], cmap='gray')
ax1.set_title('Imagen original')
ax2.imshow(edges[2], cmap='gray')
ax2.set_title('Bordes detectados')
plt.show()

#### Entrenamiento del modelo

In [ ]:
input_img = Input(shape=(512, 512, 1))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
X_train, X_val, X_test = np.split(edges, [int(.7*len(data)), int(.9*len(data))])
# y_train, y_val, y_test = np.split(labels, [int(.7*len(labels)), int(.9*len(labels))])

# y_train = np.expand_dims(y_train, axis=3)
# y_val = np.expand_dims(y_val, axis=3)
# y_test = np.expand_dims(y_test, axis=3)

In [ ]:
print(y_train.shape)

In [ ]:
input_img = Input(shape=(512, 512, 3))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
batch1 = BatchNormalization()(conv1)
maxpool1 = MaxPooling2D(pool_size=(3,3))(batch1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(maxpool1)
batch2 = BatchNormalization()(conv2)
maxpool2 = MaxPooling2D(pool_size=(2,2))(batch2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(maxpool2)
batch3 = BatchNormalization()(conv3)
maxpool3 = MaxPooling2D(pool_size=(2,2))(batch3)    

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(maxpool3)
batch4 = BatchNormalization()(conv4)

convT1 = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(batch4)
concat1 = Concatenate(axis=3)([convT1, batch4])
conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat1)

upsam1 = UpSampling2D(size=(2,2))(conv5)
convT2 = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(upsam1)
concat2 = Concatenate(axis=3)([convT2, batch3])
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat2)

upsam2 = UpSampling2D(size=(2,2))(conv6)
convT3 = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(upsam2)
concat3 = Concatenate(axis=3)([convT3, batch2])
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(concat3)

upsam3 = UpSampling2D(size=(3,3))(conv7)
convT4 = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(upsam3)
concat4 = Concatenate(axis=3)([convT4, batch1])
conv8 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(concat4)

unet = Model(input_img, conv8)
unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()]) #['accuracy', 'mse', 'mae', 'mape'])


In [ ]:
unet.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val))